Import stuff and initialize global parameters.

In [ ]:
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from keras.layers import Conv2D, Flatten, Dense, LSTM, ConvLSTM2D, \
    MaxPooling2D
from keras.models import Sequential
from common import extract_training_data

IMAGE_SIZE = (64, 64, 3)

Using TensorFlow backend.


# 2D Convolutional LSTM network
Reading data and creating initial model.

In [ ]:
images, labels = extract_training_data("Data/20171029-201949.h264.avi", 
                                       "Data/20171029-201949.h264.csv", 
                                       IMAGE_SIZE)

In [ ]:
y_train = labels.as_matrix(columns=labels.columns[1: -1])
y_train.shape

In [ ]:
images.shape

In [ ]:
from keras.layers import Permute, Reshape

model = Sequential()

model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal", activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPooling2D((2, 2)))

model.add(ConvLSTM2D(data_format='channels_last', 
                     filters=32, kernel_size=(3, 3), 
                     return_sequences=False, 
                     recurrent_activation='relu'))
model.add(Dense(3, activation='linear'))

model.compile(loss="mse", optimizer="adam")
model.summary()

### Convolutional LSTM network results

In [ ]:
history = model.fit(images, y_train, batch_size=64, epochs=10, validation_split=0.04)

In [ ]:
test_images, test_labels = extract_training_data("Data/20171029-201639.h264.avi", 
                                                 "Data/20171029-201639.h264.csv", 
                                                 IMAGE_SIZE)

prediction = model.predict(test_images)

In [ ]:
prediction

In [ ]:
plt.hist(prediction[:, 0])
plt.show()

In [ ]:
plt.hist(prediction[:, 1])
plt.show()

In [ ]:
plt.hist(prediction[:, 2])
plt.show()

In [ ]:
validation_labels = test_labels.as_matrix(columns=test_labels.columns[1: -1])
clipped_prediction = np.zeros_like(prediction)

clipped_prediction[:, 0] = np.clip(prediction[:, 0], -1, 1)
clipped_prediction[:, 1] = np.clip(prediction[:, 1], 0, 1)
clipped_prediction[:, 2] = np.clip(prediction[:, 2], 0, 1)

print(validation_labels.shape)
print(clipped_prediction.shape)

In [ ]:
from sklearn.metrics import explained_variance_score
explained_variance_score(validation_labels, prediction)

In [ ]:
from keras.models import load_model

model.save('conv_lstm.h5')

## K fold cross validation

Currently the model is just a copy of what is above, defined as a function for the KerasRegressor wrapper

In [ ]:
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import KFold, cross_val_score

def create_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), kernel_initializer="he_normal", activation='relu', input_shape=(64, 64, 3)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Permute((3, 2, 1)))
    model.add(Reshape((4, 7688)))
    model.add(LSTM(64, return_sequences=False))
    model.add(Dense(3, activation='linear'))

    model.compile(loss="mse", optimizer="adam")
    
    return model

# Concatenate both videos
total_images = np.concatenate((images, test_images), axis=0)
total_labels = np.concatenate((labels, test_labels), axis=0)

total_y_train = total_labels[:, 1:-1]

model = KerasRegressor(build_fn=create_model, epochs=10, batch_size=64, verbose=0)
kfold = KFold(n_splits=10)

results = cross_val_score(model, total_images, total_y_train, cv=kfold, scoring='explained_variance')
print(results)
print(results.mean())
print(np.median(results))